In [4]:
!pip install torchsummary

In [7]:
import torch
import torch.nn as nn
from torchsummary import summary

class YourModelClass:
    def __init__(self, config):
        self.config = config

    def model(self):
        cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (64, 75, 75)

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (128, 37, 37)

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (256, 18, 18)

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (512, 9, 9)

            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (1024, 4, 4)

            nn.Flatten()
        )

        # Assuming the input size is (3, 150, 150), the feature map size after the last MaxPool2d will be (1024, 4, 4)
        # Calculating the number of features for the fully connected layer
        num_features = 1024 * 4 * 4

        fc_layers = nn.Sequential(
            nn.Linear(num_features, 512),  # Adjusted to 512 units
            nn.ReLU(),
            nn.Linear(512, self.config.classes)
        )

        model = nn.Sequential(
            cnn,
            fc_layers
        )

        # Freezing the layers
        for param in model.parameters():
            param.requires_grad = False

        # Unfreezing layers to match the target of 5,768,454 trainable params
        # Unfreezing the last 5 convolutional layers and the fully connected layers
        for layer in model[0][-5:]:
            for param in layer.parameters():
                param.requires_grad = True

        for param in model[1].parameters():
            param.requires_grad = True

        return model

    def print_summary(self, input_size):
        model = self.model()
        summary(model, input_size)

# Example configuration
class Config:
    def __init__(self):
        self.classes = 10  # Example number of classes
        self.epsilon = 1e-8  # Epsilon for numerical stability
        self.learning_rate = 1e-3  # Learning rate
        self.weight_decay = 1e-5  # Weight decay for regularization
        self.decay_rate = 0.1  # Decay rate for learning rate

# Example usage
config = Config()
model_instance = YourModelClass(config)
model_instance.print_summary((3, 150, 150))  # Example input size (3 channels, 150x150 image)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           1,792
       BatchNorm2d-2         [-1, 64, 150, 150]             128
              ReLU-3         [-1, 64, 150, 150]               0
         MaxPool2d-4           [-1, 64, 75, 75]               0
            Conv2d-5          [-1, 128, 75, 75]          73,856
       BatchNorm2d-6          [-1, 128, 75, 75]             256
              ReLU-7          [-1, 128, 75, 75]               0
         MaxPool2d-8          [-1, 128, 37, 37]               0
            Conv2d-9          [-1, 256, 37, 37]         295,168
      BatchNorm2d-10          [-1, 256, 37, 37]             512
             ReLU-11          [-1, 256, 37, 37]               0
        MaxPool2d-12          [-1, 256, 18, 18]               0
           Conv2d-13          [-1, 512, 18, 18]       1,180,160
      BatchNorm2d-14          [-1, 512,